In [1]:
# Taken from https://www.kaggle.com/code/suvroo/complete-nlp-pipeline#RoPE-(Robust-Positional-Embeddings)

from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score,confusion_matrix
import re
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


df = pd.read_csv('IMDB Dataset.csv')
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [2]:
df.duplicated().sum()

418

In [3]:
df.drop_duplicates(inplace=True)

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/saarbu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:

def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text
df['review'] = df['review'].apply(remove_tags)
df_post_tags = df.copy()

sw_list = stopwords.words('english')
df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

df['review'] = df['review'].apply(lambda x:x.lower())

In [6]:
# df = df.sample(1000)

In [7]:
# from transformers import AutoTokenizer, AutoModel
# import torch
# import torch.nn as nn
# import pandas as pd
# from torch.utils.data import Dataset, DataLoader
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, accuracy_score
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np
# import time

# # Define the model name
# model_name = "tabularisai/multilingual-sentiment-analysis"

# # Load the tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# # Load the base model
# base_model = AutoModel.from_pretrained(model_name)

# # Check if a GPU is available and set the device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# # Define a custom model with a new classification head
# class BinarySentimentModel(nn.Module):
#     def __init__(self, base_model, num_labels=2):
#         super(BinarySentimentModel, self).__init__()
#         self.base_model = base_model
#         hidden_size = base_model.config.hidden_size
#         self.dropout = nn.Dropout(0.1)
#         self.classifier = nn.Linear(hidden_size, num_labels)

#     def forward(self, input_ids, attention_mask=None, **kwargs):
#         outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
#         pooled_output = outputs.last_hidden_state[:, 0, :]
#         pooled_output = self.dropout(pooled_output)
#         logits = self.classifier(pooled_output)
#         return logits

# # Define dataset class
# class SentimentDataset(Dataset):
#     def __init__(self, texts, labels, tokenizer, max_length=512):
#         self.texts = texts
#         self.labels = labels
#         self.tokenizer = tokenizer
#         self.max_length = max_length

#     def __len__(self):
#         return len(self.texts)

#     def __getitem__(self, idx):
#         text = str(self.texts[idx])
#         label = self.labels[idx]
#         encoding = self.tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=self.max_length)
#         return {
#             'input_ids': encoding['input_ids'].squeeze(0),
#             'attention_mask': encoding['attention_mask'].squeeze(0),
#             'labels': torch.tensor(label, dtype=torch.long)
#         }

# # Preprocess data
# df_post_tags['sentiment'] = df_post_tags['sentiment'].str.lower()
# df_post_tags['label'] = df_post_tags['sentiment'].map({'positive': 1, 'negative': 0})
# df_post_tags = df_post_tags.dropna(subset=['label'])
# df_post_tags['label'] = df_post_tags['label'].astype(int)

# # Split off a fixed validation set (10% of data)
# train_val_df, val_df = train_test_split(df_post_tags, test_size=0.1, random_state=42)
# val_texts = val_df['review'].tolist()
# val_labels = val_df['label'].tolist()
# val_dataset = SentimentDataset(val_texts, val_labels, tokenizer)
# val_loader = DataLoader(val_dataset, batch_size=16)

# # Use exactly 5000 samples for training
# train_size = 5000
# print(f"\nTraining with {train_size} samples...")

# # Subsample training data
# train_df_sample = train_val_df.sample(n=min(train_size, len(train_val_df)), random_state=42)
# train_texts = train_df_sample['review'].tolist()
# train_labels = train_df_sample['label'].tolist()

# # Create training dataset and loader
# train_dataset = SentimentDataset(train_texts, train_labels, tokenizer)
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# # Initialize model
# model = BinarySentimentModel(base_model).to(device)
# optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
# loss_fn = nn.CrossEntropyLoss()

# # Store training history
# history = {
#     'train_loss': [],
#     'val_loss': [],
#     'val_acc': []
# }

# # Training loop with validation at each epoch
# print("Starting training...")
# start_time = time.time()
# num_epochs = 3
# for epoch in range(num_epochs):
#     # Training
#     model.train()
#     total_train_loss = 0
#     for batch in train_loader:
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)

#         optimizer.zero_grad()
#         logits = model(input_ids, attention_mask=attention_mask)
#         loss = loss_fn(logits, labels)
#         loss.backward()
#         optimizer.step()
#         total_train_loss += loss.item()
    
#     avg_train_loss = total_train_loss / len(train_loader)
#     history['train_loss'].append(avg_train_loss)
    
#     # Validation
#     model.eval()
#     val_loss = 0
#     correct = 0
#     total = 0
#     with torch.no_grad():
#         for batch in val_loader:
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             labels = batch['labels'].to(device)
#             logits = model(input_ids, attention_mask=attention_mask)
#             loss = loss_fn(logits, labels)
#             val_loss += loss.item()
#             preds = torch.argmax(logits, dim=-1)
#             correct += (preds == labels).sum().item()
#             total += labels.size(0)
    
#     avg_val_loss = val_loss / len(val_loader)
#     accuracy = correct / total
    
#     # Store results
#     history['val_loss'].append(avg_val_loss)
#     history['val_acc'].append(accuracy)
    
#     print(f"Epoch {epoch + 1} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Val Accuracy: {accuracy:.4f}")

# training_time = time.time() - start_time
# print(f"Training completed in {training_time:.2f} seconds")

# # Save the model
# torch.save(model.state_dict(), "sentiment_model_5000_samples.pt")
# print("Model saved as 'sentiment_model_5000_samples.pt'")

# # Plot training results
# plt.figure(figsize=(15, 5))

# # Plot Training and Validation Loss
# plt.subplot(1, 2, 1)
# plt.plot(range(1, num_epochs + 1), history['train_loss'], marker='o', label="Training Loss")
# plt.plot(range(1, num_epochs + 1), history['val_loss'], marker='o', label="Validation Loss")
# plt.xlabel("Epoch")
# plt.ylabel("Loss")
# plt.title("Training and Validation Loss")
# plt.legend()
# plt.grid(True)

# # Plot Validation Accuracy
# plt.subplot(1, 2, 2)
# plt.plot(range(1, num_epochs + 1), history['val_acc'], marker='o', color='green')
# plt.xlabel("Epoch")
# plt.ylabel("Accuracy")
# plt.title("Validation Accuracy")
# plt.grid(True)

# plt.tight_layout()
# plt.savefig("training_results.png")
# plt.show()

# # ========================
# # Evaluate on all 50k samples
# # ========================
# print("\nEvaluating model on all 50k samples...")
# eval_start_time = time.time()

# # Assume X contains all 50k text samples and y contains the corresponding labels
# # Create a dataset for the full 50k samples
# full_dataset = SentimentDataset(X, y, tokenizer)
# batch_size = 32  # Adjust based on your GPU memory
# full_loader = DataLoader(full_dataset, batch_size=batch_size)

# # Evaluate on the full dataset
# model.eval()
# all_preds = []
# all_probs = []  # Store probabilities for ROC AUC calculation
# all_labels = []

# with torch.no_grad():
#     for batch in full_loader:
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)
        
#         logits = model(input_ids, attention_mask=attention_mask)
#         probs = torch.softmax(logits, dim=1)
#         preds = torch.argmax(logits, dim=1)
        
#         all_preds.extend(preds.cpu().numpy())
#         all_probs.extend(probs[:, 1].cpu().numpy())  # Probability for positive class
#         all_labels.extend(labels.cpu().numpy())

# eval_time = time.time() - eval_start_time
# print(f"Evaluation completed in {eval_time:.2f} seconds")

# # Convert predictions and probabilities to numpy arrays
# all_preds = np.array(all_preds)
# all_probs = np.array(all_probs)
# all_labels = np.array(all_labels)

# # Calculate metrics
# accuracy = accuracy_score(all_labels, all_preds)
# print(f"\nAccuracy on all 50k samples: {accuracy:.4f}")

# # Calculate AUC-ROC 
# # Note: For binary classification, we use the probability of the positive class
# auc_roc = roc_auc_score(all_labels, all_probs)
# print(f"AUC-ROC on all 50k samples: {auc_roc:.4f}")

# # Get the original class labels using the label encoder
# pred_classes = le.inverse_transform(all_preds)
# true_classes = le.inverse_transform(all_labels)

# # Print classification report
# print("\nClassification Report:")
# print(classification_report(true_classes, pred_classes))

# # Create confusion matrix
# cm = confusion_matrix(true_classes, pred_classes)
# class_names = le.classes_

# # Plot confusion matrix
# plt.figure(figsize=(10, 8))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
# plt.xlabel('Predicted')
# plt.ylabel('True')
# plt.title('Confusion Matrix')
# plt.savefig("confusion_matrix.png")
# plt.show()

# # Plot ROC curve
# fpr, tpr, _ = roc_curve(all_labels, all_probs)
# plt.figure(figsize=(8, 6))
# plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {auc_roc:.4f})')
# plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('ROC Curve')
# plt.legend(loc='lower right')
# plt.grid(True)
# plt.savefig("roc_curve.png")
# plt.show()

# # Per-class metrics for multiclass classification (if applicable)
# if len(class_names) > 2:
#     # Calculate per-class AUC-ROC using one-vs-rest approach
#     from sklearn.preprocessing import label_binarize
#     from sklearn.metrics import roc_auc_score
    
#     # Binarize the labels for multi-class ROC AUC
#     y_bin = label_binarize(all_labels, classes=range(len(class_names)))
    
#     # Get model predictions with probabilities for each class
#     model.eval()
#     all_class_probs = []
    
#     with torch.no_grad():
#         for batch in full_loader:
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
            
#             logits = model(input_ids, attention_mask=attention_mask)
#             probs = torch.softmax(logits, dim=1).cpu().numpy()
#             all_class_probs.append(probs)
    
#     all_class_probs = np.vstack(all_class_probs)
    
#     # Calculate AUC for each class
#     roc_auc = {}
#     for i in range(len(class_names)):
#         roc_auc[class_names[i]] = roc_auc_score(y_bin[:, i], all_class_probs[:, i])
    
#     print("\nAUC-ROC per class:")
#     for cls, auc in roc_auc.items():
#         print(f"{cls}: {auc:.4f}")

# # Display a few sample predictions
# print("\nSample Predictions:")
# sample_indices = np.random.choice(len(X), min(10, len(X)), replace=False)
# for idx in sample_indices:
#     text = X[idx]
#     true_label = le.inverse_transform([all_labels[idx]])[0]
#     pred_label = le.inverse_transform([all_preds[idx]])[0]
#     prob = all_probs[idx]
    
#     print(f"Text: '{text[:100]}...'")
#     print(f"True label: {true_label}")
#     print(f"Predicted label: {pred_label} (confidence: {prob:.4f})")
#     print("-" * 50)

In [8]:
# encoder = LabelEncoder()
# y = encoder.fit_transform(y)

In [9]:
# X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)
# X_train.shape

In [10]:
# # Applying BoW
# cv = CountVectorizer()
# X_train_bow = cv.fit_transform(X_train['review'])
# X_test_bow = cv.transform(X_test['review'])

# X_train_bow.shape

In [11]:
# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier()

# rf.fit(X_train_bow,y_train)
# y_pred = rf.predict(X_test_bow)
# accuracy_score(y_test,y_pred)

In [12]:
# X_train_bow

In [13]:
# import optuna
# import numpy as np
# import xgboost as xgb
# from sklearn.model_selection import RepeatedKFold

# from sklearn.metrics import roc_auc_score

# import xgboost as xgb
# import numpy as np
# from sklearn.metrics import roc_auc_score
# from sklearn.model_selection import train_test_split

# from sklearn.model_selection import StratifiedKFold

# def objective(trial):
#     params = {
#         'n_estimators': trial.suggest_int('n_estimators', 50, 1000, step=50),
#         "eta": trial.suggest_float("eta", 0.01, 0.3, log=True),
#         "max_depth": trial.suggest_int("max_depth", 3, 20),
#         "subsample": trial.suggest_float("subsample", 0.5, 1.0),
#         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
#         "lambda": trial.suggest_float("lambda", 1e-3, 10.0, log=True),
#         "alpha": trial.suggest_float("alpha", 1e-3, 10.0, log=True),
#         "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
#         "gamma": trial.suggest_float("gamma", 0, 5),
#         "objective": "binary:logistic",
#         "eval_metric": "auc",
#         "verbosity": 0,
#         "tree_method": "hist",
#         "device": "cuda"
#     }

#     cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#     aucs = []

#     for train_idx, val_idx in cv.split(X_train_bow, y_train):
#         X_tr, X_val = X_train_bow[train_idx], X_train_bow[val_idx]
#         y_tr, y_val = y_train[train_idx], y_train[val_idx]

#         dtrain = xgb.DMatrix(X_tr, label=y_tr)
#         dvalid = xgb.DMatrix(X_val, label=y_val)

#         model = xgb.train(
#             params=params,
#             dtrain=dtrain,
#             evals=[(dvalid, "eval")],
#             early_stopping_rounds=30,
#             verbose_eval=False
#         )

#         preds = model.predict(dvalid, iteration_range=(0, model.best_iteration))
#         auc = roc_auc_score(y_val, preds)
#         aucs.append(auc)

#     return np.mean(aucs)


# # Run Optuna optimization
# pruner = optuna.pruners.HyperbandPruner()
# study = optuna.create_study(direction="maximize",pruner=pruner)

# # Define baseline trial as a fixed parameter suggestion
# baseline_params = {
#     'n_estimators': 550,
#     'eta': 0.29983309207957676,
#     'max_depth': 19,
#     'subsample': 0.9888438323886279,
#     'colsample_bytree': 0.6889387283870231,
#     'lambda': 1.0481576423947685,
#     'alpha': 0.023164905855125376,
#     'min_child_weight': 10,
#     'gamma': 3.4009242038001064,
# }

# # Add the trial to the queue to run
# study.enqueue_trial(baseline_params)

# study.optimize(objective,timeout=3600)

# # Best hyperparameters found
# best_params = study.best_params
# print(f"\n🔹 Best Hyperparameters:\n{best_params}")

# # Train final model with best hyperparameters on GPU
# best_params["tree_method"] = "gpu_hist"
# best_params["device"] = "cuda"

# dtrain_cv = xgb.DMatrix(X_train_bow, label=y_train)
# final_model = xgb.train(
#     params=best_params,
#     dtrain=dtrain_cv,
# )

# # Evaluate on Hold-Out Validation Set
# dvalid = xgb.DMatrix(X_test_bow, label=y_test)
# test_preds = final_model.predict(dvalid)

# # Convert probabilities to binary labels
# pred_labels = (test_preds > 0.5).astype(int)

# accuracy_score(y_test,pred_labels)


# {'n_estimators': 300, 'eta': 0.29945643753119816, 'max_depth': 20, 'subsample': 0.8783578562355324, 'colsample_bytree': 0.7338100018725511, 'lambda': 0.03509555712821362, 'alpha': 0.0029994420277393804, 'min_child_weight': 10, 'gamma': 2.7368297453515065}

In [14]:
# # Evaluate on Hold-Out Validation Set
# dvalid = xgb.DMatrix(X_test_bow, label=y_test)
# test_preds = final_model.predict(dvalid)


# accuracy_score(y_test,pred_labels)

# Finetuning on different amount of samples for performance comparison

In [16]:
# from transformers import AutoTokenizer, AutoModel
# import torch
# import torch.nn as nn
# import pandas as pd
# from torch.utils.data import Dataset, DataLoader
# from sklearn.model_selection import train_test_split
# import copy
# import matplotlib.pyplot as plt

# # Define the model name
# model_name = "tabularisai/multilingual-sentiment-analysis"

# # Load the tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# # Load the base model
# base_model = AutoModel.from_pretrained(model_name)

# # Check if a GPU is available and set the device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# # Define a custom model with a new classification head
# class BinarySentimentModel(nn.Module):
#     def __init__(self, base_model, num_labels=2):
#         super(BinarySentimentModel, self).__init__()
#         self.base_model = base_model
#         hidden_size = base_model.config.hidden_size
#         self.dropout = nn.Dropout(0.1)
#         self.classifier = nn.Linear(hidden_size, num_labels)

#     def forward(self, input_ids, attention_mask=None, **kwargs):
#         outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
#         pooled_output = outputs.last_hidden_state[:, 0, :]
#         pooled_output = self.dropout(pooled_output)
#         logits = self.classifier(pooled_output)
#         return logits

# # Define dataset class
# class SentimentDataset(Dataset):
#     def __init__(self, texts, labels, tokenizer, max_length=512):
#         self.texts = texts
#         self.labels = labels
#         self.tokenizer = tokenizer
#         self.max_length = max_length

#     def __len__(self):
#         return len(self.texts)

#     def __getitem__(self, idx):
#         text = str(self.texts[idx])
#         label = self.labels[idx]
#         encoding = self.tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=self.max_length)
#         return {
#             'input_ids': encoding['input_ids'].squeeze(0),
#             'attention_mask': encoding['attention_mask'].squeeze(0),
#             'labels': torch.tensor(label, dtype=torch.long)
#         }

# # Preprocess data
# df_post_tags['sentiment'] = df_post_tags['sentiment'].str.lower()
# df_post_tags['label'] = df_post_tags['sentiment'].map({'positive': 1, 'negative': 0})
# df_post_tags = df_post_tags.dropna(subset=['label'])
# df_post_tags['label'] = df_post_tags['label'].astype(int)

# # Split off a fixed validation set (10% of 50k = 5k samples)
# train_val_df, val_df = train_test_split(df_post_tags, test_size=0.1, random_state=42)
# val_texts = val_df['review'].tolist()
# val_labels = val_df['label'].tolist()
# val_dataset = SentimentDataset(val_texts, val_labels, tokenizer)
# val_loader = DataLoader(val_dataset, batch_size=16)

# # Define training sizes to test
# train_sizes = [100, 250, 500, 1000, 2500, 5000, 10000, 25000]

# # Store results (accuracy and loss per epoch for each size)
# results = {size: {'val_loss': [], 'val_acc': []} for size in train_sizes}

# # Training and evaluation loop for each size
# for size in train_sizes:
#     print(f"\nTraining with {size} samples...")
    
#     # Subsample training data
#     train_df_sample = train_val_df.sample(n=min(size, len(train_val_df)), random_state=42)
#     train_texts = train_df_sample['review'].tolist()
#     train_labels = train_df_sample['label'].tolist()
    
#     # Create training dataset and loader
#     train_dataset = SentimentDataset(train_texts, train_labels, tokenizer)
#     train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    
#     # Reset model
#     model = BinarySentimentModel(copy.deepcopy(base_model)).to(device)
#     optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
#     loss_fn = nn.CrossEntropyLoss()
    
#     # Training loop with validation at each epoch
#     num_epochs = 3
#     for epoch in range(num_epochs):
#         # Training
#         model.train()
#         total_train_loss = 0
#         for batch in train_loader:
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             labels = batch['labels'].to(device)

#             optimizer.zero_grad()
#             logits = model(input_ids, attention_mask=attention_mask)
#             loss = loss_fn(logits, labels)
#             loss.backward()
#             optimizer.step()
#             total_train_loss += loss.item()
        
#         avg_train_loss = total_train_loss / len(train_loader)
        
#         # Validation
#         model.eval()
#         val_loss = 0
#         correct = 0
#         total = 0
#         with torch.no_grad():
#             for batch in val_loader:
#                 input_ids = batch['input_ids'].to(device)
#                 attention_mask = batch['attention_mask'].to(device)
#                 labels = batch['labels'].to(device)
#                 logits = model(input_ids, attention_mask=attention_mask)
#                 loss = loss_fn(logits, labels)
#                 val_loss += loss.item()
#                 preds = torch.argmax(logits, dim=-1)
#                 correct += (preds == labels).sum().item()
#                 total += labels.size(0)
        
#         avg_val_loss = val_loss / len(val_loader)
#         accuracy = correct / total
        
#         # Store results
#         results[size]['val_loss'].append(avg_val_loss)
#         results[size]['val_acc'].append(accuracy)
        
#         print(f"Epoch {epoch + 1} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Val Accuracy: {accuracy:.4f}")

# # Print final results
# print("\nFinal Results:")
# for size in train_sizes:
#     print(f"Training Size: {size}")
#     for epoch in range(num_epochs):
#         print(f"  Epoch {epoch + 1} - Val Loss: {results[size]['val_loss'][epoch]:.4f}, Val Accuracy: {results[size]['val_acc'][epoch]:.4f}")

# # Optional: Plot results
# plt.figure(figsize=(12, 5))

# # Plot Validation Loss
# plt.subplot(1, 2, 1)
# for size in train_sizes:
#     plt.plot(range(1, num_epochs + 1), results[size]['val_loss'], marker='o', label=f"Size {size}")
# plt.xlabel("Epoch")
# plt.ylabel("Validation Loss")
# plt.title("Validation Loss vs. Epoch")
# plt.legend()
# plt.grid(True)

# # Plot Validation Accuracy
# plt.subplot(1, 2, 2)
# for size in train_sizes:
#     plt.plot(range(1, num_epochs + 1), results[size]['val_acc'], marker='o', label=f"Size {size}")
# plt.xlabel("Epoch")
# plt.ylabel("Validation Accuracy")
# plt.title("Validation Accuracy vs. Epoch")
# plt.legend()
# plt.grid(True)

# plt.tight_layout()
# plt.show()

# Review length analysis in tokens (for bert)

In [17]:
# from transformers import AutoTokenizer, AutoModel
# import torch
# import torch.nn as nn
# import pandas as pd
# from torch.utils.data import Dataset, DataLoader
# from sklearn.model_selection import train_test_split
# import copy
# import matplotlib.pyplot as plt

# # Define the model name
# model_name = "tabularisai/multilingual-sentiment-analysis"

# # Load the tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# # Load the base model
# base_model = AutoModel.from_pretrained(model_name)

# # Check if a GPU is available and set the device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# from transformers import AutoTokenizer, AutoModel
# import torch
# import torch.nn as nn
# import pandas as pd
# from torch.utils.data import Dataset, DataLoader
# from sklearn.model_selection import train_test_split
# import copy
# import matplotlib.pyplot as plt
# import numpy as np

# # Define the model name
# model_name = "tabularisai/multilingual-sentiment-analysis"

# # Load the tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# # Load the base model
# base_model = AutoModel.from_pretrained(model_name)

# # Check if a GPU is available and set the device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# # Preprocess data
# df_post_tags['sentiment'] = df_post_tags['sentiment'].str.lower()
# df_post_tags['label'] = df_post_tags['sentiment'].map({'positive': 1, 'negative': 0})
# df_post_tags = df_post_tags.dropna(subset=['label'])
# df_post_tags['label'] = df_post_tags['label'].astype(int)

# # Analyze review lengths in terms of tokens
# def get_token_lengths(texts, tokenizer):
#     token_lengths = []
#     for text in texts:
#         tokens = tokenizer(text, truncation=False, return_tensors="pt", add_special_tokens=True)['input_ids'][0]
#         token_lengths.append(len(tokens))
#     return token_lengths

# # Compute token lengths for the 'review' column
# review_texts = df_post_tags['review'].tolist()
# token_lengths = get_token_lengths(review_texts, tokenizer)

# # Calculate max, median, and other stats
# max_length = max(token_lengths)
# median_length = int(np.median(token_lengths))
# total_reviews = len(token_lengths)

# print(f"Review Length Analysis (in tokens):")
# print(f"  Total Reviews: {total_reviews}")
# print(f"  Maximum Token Length: {max_length}")
# print(f"  Median Token Length: {median_length}")

# # Optional: Additional stats for deeper insight
# percentile_90 = int(np.percentile(token_lengths, 90))
# print(f"  90th Percentile Token Length: {percentile_90}")

# Finetuning on 10k samples and saving the rest

In [28]:
%%time
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import copy
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModel
import os

NUM_EPOCHS = 2
BATCH_SIZE = 64
LEARNING_RATE = 2e-5
MODEL_SAVE_PATH = "fine_tuned_sentiment_model_10k_samples.pth"
TRAIN_SAMPLES_SAVE_PATH = "train_10k_samples.csv"      # Path to save training samples
VAL_SAMPLES_SAVE_PATH = "validation_samples.csv"      # Path to save validation samples
OTHER_SAMPLES_SAVE_PATH = "test_set.csv" # Path to save samples not in train or val


# Define the model name
model_name = "tabularisai/multilingual-sentiment-analysis"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the base model
base_model = AutoModel.from_pretrained(model_name)

# Check if a GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define a custom model with a new classification head
class BinarySentimentModel(nn.Module):
    def __init__(self, base_model, num_labels=2):
        super(BinarySentimentModel, self).__init__()
        self.base_model = base_model
        hidden_size = base_model.config.hidden_size
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None, **kwargs):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :] # Assuming [CLS] token
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

# Define dataset class
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=52): # Adjusted max_length
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=self.max_length
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# --- Data Loading and Preprocessing (Assuming df_post_tags is available) ---
# If df_post_tags is not already loaded, load it here.
# Example:
# df_post_tags = pd.read_csv("your_data.csv") # Replace with your data loading

# Placeholder for df_post_tags. Replace with your actual data loading.
# Ensure df_post_tags is loaded before this point.
# For demonstration, creating a dummy DataFrame:
if 'df_post_tags' not in locals():
    print("Creating dummy df_post_tags. Replace with your actual data loading.")
    data = {'review': [f'This is a positive review {i}.' for i in range(25000)] + [f'This is a negative review {i}.' for i in range(25000)],
            'sentiment': ['positive'] * 25000 + ['negative'] * 25000}
    df_post_tags = pd.DataFrame(data)
    df_post_tags = df_post_tags.sample(frac=1, random_state=42).reset_index(drop=True) # Shuffle


# Add a unique identifier before any splitting/sampling
df_post_tags['original_index'] = df_post_tags.index

df_post_tags['sentiment'] = df_post_tags['sentiment'].str.lower()
df_post_tags['label'] = df_post_tags['sentiment'].map({'positive': 1, 'negative': 0})
df_post_tags = df_post_tags.dropna(subset=['label'])
df_post_tags['label'] = df_post_tags['label'].astype(int)

# --- Split Data ---
# Split off a fixed validation set (e.g., 10% of the available data)
train_val_df, val_df = train_test_split(df_post_tags, test_size=0.1, random_state=42)

# Define the training size
TRAIN_SIZE = 10000

# Subsample training data to exactly TRAIN_SIZE samples
if len(train_val_df) > TRAIN_SIZE:
    train_df_sample = train_val_df.sample(n=TRAIN_SIZE, random_state=42) # Do NOT reset index here yet
    print(f"Selected {len(train_df_sample)} samples for training from {len(train_val_df)} available.")
else:
    train_df_sample = train_val_df # Use all available if less than TRAIN_SIZE
    TRAIN_SIZE = len(train_df_sample) # Update TRAIN_SIZE to actual size
    print(f"Warning: Only {len(train_val_df)} samples available for training, using all ({TRAIN_SIZE}).")


# --- Identify and Save Data Splits ---

# Get the 'original_index' of the samples used for training and validation
used_train_indices = train_df_sample['original_index']
used_val_indices = val_df['original_index'] # Get indices for the validation set

# Filter original_df to get samples NOT in train or validation
# Start with the original DataFrame and remove indices present in train_df_sample or val_df
all_indices = df_post_tags['original_index']
train_val_indices = pd.concat([used_train_indices, used_val_indices])
other_samples_df = df_post_tags[~all_indices.isin(train_val_indices)]


try:
    # Save the training samples
    train_df_sample_saved = train_df_sample.copy().drop(columns=['original_index']) # Drop original_index before saving if not needed
    train_df_sample_saved.to_csv(TRAIN_SAMPLES_SAVE_PATH, index=False)
    print(f"\nSaved {len(train_df_sample_saved)} training samples to {TRAIN_SAMPLES_SAVE_PATH}")

    # Save the validation samples
    val_df_saved = val_df.copy().drop(columns=['original_index']) # Drop original_index before saving if not needed
    val_df_saved.to_csv(VAL_SAMPLES_SAVE_PATH, index=False)
    print(f"Saved {len(val_df_saved)} validation samples to {VAL_SAMPLES_SAVE_PATH}")

    # Save the other remaining samples
    other_samples_df_saved = other_samples_df.copy().drop(columns=['original_index']) # Drop original_index before saving if not needed
    other_samples_df_saved.to_csv(OTHER_SAMPLES_SAVE_PATH, index=False)
    print(f"Saved {len(other_samples_df_saved)} other remaining samples (not train or val) to {OTHER_SAMPLES_SAVE_PATH}")


except Exception as e:
    print(f"Error saving data splits: {e}")


# --- Prepare Training Data (Now Reset Index for Dataset) ---
# Reset index *after* identifying unused samples and saving train_df_sample
# The saved train_df_sample retains the original_index if you need it later, but we use the reset version for the DataLoader
train_df_sample = train_df_sample.reset_index(drop=True)

train_texts = train_df_sample['review'].tolist()
train_labels = train_df_sample['label'].tolist()

# Create training dataset and loader
train_dataset = SentimentDataset(train_texts, train_labels, tokenizer)
# Use the potentially adjusted TRAIN_SIZE in the print statement
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)


# --- Prepare Validation Data (using the saved val_df) ---
# This part remains the same as val_df was already defined
val_texts = val_df['review'].tolist()
val_labels = val_df['label'].tolist()
val_dataset = SentimentDataset(val_texts, val_labels, tokenizer)

val_loader = DataLoader(val_dataset, batch_size=16)

# --- Fine-tuning on TRAIN_SIZE Samples ---
print(f"\nStarting training with {TRAIN_SIZE} samples...")

# Reset model (or initialize it for the first time in this run)
# We keep the model initialization here before the training loop
model = BinarySentimentModel(copy.deepcopy(base_model)).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss()

# Training loop with validation at each epoch
for epoch in range(NUM_EPOCHS):
    # Training
    model.train()
    total_train_loss = 0
    for step, batch in enumerate(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        logits = model(input_ids, attention_mask=attention_mask)
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

        # --- Add Validation Calculation Every 10 Steps ---
        if (step + 1) % 10 == 0: # Print progress and validate every 10 steps
            current_train_loss = loss.item() # Loss for the current batch

            # Perform Validation for this step's check
            model.eval() # Set model to evaluation mode
            step_val_loss = 0
            step_correct = 0
            step_total = 0
            with torch.no_grad():
                for val_batch in val_loader:
                    val_input_ids = val_batch['input_ids'].to(device)
                    val_attention_mask = val_batch['attention_mask'].to(device)
                    val_labels = val_batch['labels'].to(device)

                    val_logits = model(val_input_ids, attention_mask=val_attention_mask)
                    v_loss = loss_fn(val_logits, val_labels)
                    step_val_loss += v_loss.item()

                    val_preds = torch.argmax(val_logits, dim=-1)
                    step_correct += (val_preds == val_labels).sum().item()
                    step_total += val_labels.size(0)

            # Calculate average validation metrics for this step's check
            avg_step_val_loss = step_val_loss / len(val_loader)
            step_accuracy = step_correct / step_total

            # Print training loss for the current step PLUS validation metrics
            print(f"Epoch {epoch + 1}, Step {step + 1}/{len(train_loader)} | Train Loss (Step): {current_train_loss:.4f} | Val Loss (Current): {avg_step_val_loss:.4f} | Val Acc (Current): {step_accuracy:.4f}")

            model.train() # Set model back to training mode
        # --- End of Validation Calculation Every 10 Steps ---


    avg_train_loss = total_train_loss / len(train_loader)
    print(f"\nEpoch {epoch + 1} Training complete. Avg Train Loss: {avg_train_loss:.4f}")

    # Validation
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            logits = model(input_ids, attention_mask=attention_mask)
            loss = loss_fn(logits, labels)
            val_loss += loss.item()
            preds = torch.argmax(logits, dim=-1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_val_loss = val_loss / len(val_loader)
    accuracy = correct / total

    print(f"Epoch {epoch + 1} | Val Loss: {avg_val_loss:.4f} | Val Accuracy: {accuracy:.4f}")

print("\nFine-tuning finished.")

# --- Save the Fine-tuned Model ---
# It's generally best practice to save the state_dict for custom models
try:
    torch.save(model.state_dict(), MODEL_SAVE_PATH)
    print(f"Model state dictionary saved successfully to {MODEL_SAVE_PATH}")
except Exception as e:
    print(f"Error saving model: {e}")

# --- Code to Load the Saved Model ---

# To load the model, you need to first instantiate the model class
# and then load the state dictionary.

print(f"\n--- Loading the saved model from {MODEL_SAVE_PATH} ---")

# Ensure the model class and device are defined (they are in this script)
# model_name = "tabularisai/multilingual-sentiment-analysis" # Already defined
# tokenizer = AutoTokenizer.from_pretrained(model_name) # Already defined
# base_model_for_loading = AutoModel.from_pretrained(model_name) # Need a new instance or reuse
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Already defined

# Instantiate the model
loaded_model = BinarySentimentModel(AutoModel.from_pretrained(model_name)).to(device)

# Load the state dictionary
try:
    # Check if the file exists before trying to load
    if os.path.exists(MODEL_SAVE_PATH):
        loaded_model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=device))
        print("Model loaded successfully.")

        # Set the model to evaluation mode
        loaded_model.eval()
        print("Loaded model set to evaluation mode.")

        # You can now use 'loaded_model' for inference
        # Example inference (using a sample text)
        sample_text = "This is a great product!"
        encoded_input = tokenizer(sample_text, return_tensors="pt", truncation=True, padding="max_length", max_length=52).to(device)

        with torch.no_grad():
            logits = loaded_model(encoded_input['input_ids'], attention_mask=encoded_input['attention_mask'])
            probabilities = torch.softmax(logits, dim=1)
            predicted_class_id = torch.argmax(probabilities, dim=1).item()

        sentiment_map = {1: 'positive', 0: 'negative'}
        predicted_sentiment = sentiment_map.get(predicted_class_id, 'unknown')

        print(f"\nSample Inference using loaded model:")
        print(f"Text: '{sample_text}'")
        print(f"Logits: {logits}")
        print(f"Probabilities: {probabilities}")
        print(f"Predicted Class ID: {predicted_class_id}")
        print(f"Predicted Sentiment: {predicted_sentiment}")
    else:
        print(f"Error: Model file not found at {MODEL_SAVE_PATH}. Skipping model loading and inference example.")


except Exception as e:
    print(f"Error loading model: {e}")

Using device: cuda
Selected 10000 samples for training from 44623 available.

Saved 10000 training samples to train_10k_samples.csv
Saved 4959 validation samples to validation_samples.csv
Saved 34623 other remaining samples (not train or val) to other_remaining_samples.csv

Starting training with 10000 samples...
Epoch 1, Step 10/157 | Train Loss (Step): 0.5722 | Val Loss (Current): 0.5540 | Val Acc (Current): 0.7145
Epoch 1, Step 20/157 | Train Loss (Step): 0.5521 | Val Loss (Current): 0.5335 | Val Acc (Current): 0.7201
Epoch 1, Step 30/157 | Train Loss (Step): 0.5780 | Val Loss (Current): 0.5317 | Val Acc (Current): 0.7185
Epoch 1, Step 40/157 | Train Loss (Step): 0.5276 | Val Loss (Current): 0.5171 | Val Acc (Current): 0.7358
Epoch 1, Step 50/157 | Train Loss (Step): 0.6036 | Val Loss (Current): 0.5112 | Val Acc (Current): 0.7401
Epoch 1, Step 60/157 | Train Loss (Step): 0.4977 | Val Loss (Current): 0.5078 | Val Acc (Current): 0.7407
Epoch 1, Step 70/157 | Train Loss (Step): 0.4342 

<timed exec>:288: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


Model loaded successfully.
Loaded model set to evaluation mode.

Sample Inference using loaded model:
Text: 'This is a great product!'
Logits: tensor([[-2.0170,  2.0462]], device='cuda:0')
Probabilities: tensor([[0.0169, 0.9831]], device='cuda:0')
Predicted Class ID: 1
Predicted Sentiment: positive
CPU times: user 3min 14s, sys: 1min, total: 4min 14s
Wall time: 3min 44s


In [31]:
# --- Add imports if needed for this section ---
# import pandas as pd # Already imported
# import torch # Already imported
# from torch.utils.data import DataLoader # Already imported
# from torch.utils.data import Dataset # Already imported
import numpy as np # Need numpy for concatenating probabilities


# --- Code to Run Predictions on Other Remaining Samples ---

OTHER_SAMPLES_LOAD_PATH = "test_set.csv" # Path to load the data
PREDICTIONS_SAVE_PATH = "test_set_predictions.csv" # Path to save data with predictions

print(f"\n--- Running predictions on {OTHER_SAMPLES_LOAD_PATH} ---")

# Check if the loaded_model is available and in evaluation mode
if 'loaded_model' not in locals() or loaded_model is None:
    print("Error: Model is not loaded. Please run the previous code block first.")
else:
    try:
        # Load the other remaining samples
        if os.path.exists(OTHER_SAMPLES_LOAD_PATH):
            other_df = pd.read_csv(OTHER_SAMPLES_LOAD_PATH)
            print(f"Loaded {len(other_df)} samples from {OTHER_SAMPLES_LOAD_PATH}")

            # Prepare data for prediction
            prediction_texts = other_df['review'].tolist()
            # Assuming 'label' column exists from previous saving steps:
            prediction_labels = other_df['label'].tolist() # We need the true labels to calculate performance

            prediction_dataset = SentimentDataset(prediction_texts, prediction_labels, tokenizer)
            prediction_loader = DataLoader(prediction_dataset, batch_size=BATCH_SIZE, shuffle=False) # shuffle=False is important to keep order

            # Run predictions
            loaded_model.eval() # Ensure model is in evaluation mode
            all_probs = []

            print("Generating predictions...")
            with torch.no_grad():
                for step, batch in enumerate(prediction_loader):
                    input_ids = batch['input_ids'].to(device)
                    attention_mask = batch['attention_mask'].to(device)
                    # Labels are not needed for inference itself, but Dataset returns them

                    logits = loaded_model(input_ids, attention_mask=attention_mask)
                    probs = torch.softmax(logits, dim=1) # Get probabilities

                    all_probs.append(probs.cpu().numpy())

                    if (step + 1) % 100 == 0:
                         print(f"Prediction Step {step + 1}/{len(prediction_loader)}")


            # Concatenate all probabilities
            all_probs = np.concatenate(all_probs, axis=0)

            # Add probabilities to the DataFrame
            other_df['prob_negative'] = all_probs[:, 0] # Probability for class 0
            other_df['prob_positive'] = all_probs[:, 1] # Probability for class 1

            # Add predicted class (0 or 1)
            other_df['predicted_label'] = np.argmax(all_probs, axis=1)

            # --- Add Performance Calculation ---
            # Compare predicted label with the true label
            other_df['performance'] = (other_df['predicted_label'] == other_df['label']).astype(int) # 1 if correct, 0 if wrong

            # Calculate the percentage correct
            percentage_correct = other_df['performance'].mean() * 100

            print(f"\nPerformance Analysis on This Dataset:")
            print(f"Total samples: {len(other_df)}")
            print(f"Correct predictions: {other_df['performance'].sum()}")
            print(f"Incorrect predictions: {len(other_df) - other_df['performance'].sum()}")
            print(f"Accuracy on this set: {percentage_correct:.2f}%")
            # --- End of Performance Calculation ---


            # Save the DataFrame with predictions and performance
            other_df.to_csv(PREDICTIONS_SAVE_PATH, index=False)

            print(f"\nPredictions and performance saved to {PREDICTIONS_SAVE_PATH}")
            print(other_df.head()) # Print the first few rows of the result

        else:
            print(f"Error: Data file not found at {OTHER_SAMPLES_LOAD_PATH}. Skipping prediction.")

    except Exception as e:
        print(f"An error occurred during prediction: {e}")


--- Running predictions on test_set.csv ---
Loaded 34623 samples from test_set.csv
Generating predictions...
Prediction Step 100/541
Prediction Step 200/541
Prediction Step 300/541
Prediction Step 400/541
Prediction Step 500/541

Performance Analysis on This Dataset:
Total samples: 34623
Correct predictions: 26526
Incorrect predictions: 8097
Accuracy on this set: 76.61%

Predictions and performance saved to test_set_predictions.csv
                                              review sentiment  label  \
0  One of the other reviewers has mentioned that ...  positive      1   
1  I thought this was a wonderful way to spend ti...  positive      1   
2  Basically there's a family where a little boy ...  negative      0   
3  I sure would like to see a resurrection of a u...  positive      1   
4  This show was an amazing, fresh & innovative i...  negative      0   

   prob_negative  prob_positive  predicted_label  performance  
0       0.668688       0.331312                0            